# SQL para Engenharia de Dados e Ciência de Dados - Aula 7: Subqueries, Particionamento e Visões

## **Técnicas avançadas de consulta para otimização e organização de dados**

---

## **1. Subqueries**

### **1.1 O que são Subqueries?**
Subqueries (ou subconsultas) permitem inserir uma consulta SQL dentro de outra consulta. 
A subquery, ou consulta interna, é usada como uma condição ou parte da consulta principal (main query).

**Regras importantes:**
- A subquery deve ser escrita entre parênteses.
- A subquery é executada antes da main query.
- Ela pode ser usada com os comandos `SELECT`, `INSERT`, `UPDATE` ou `DELETE`.
- Subqueries funcionam com operadores como `=`, `>`, `<=`, `>=`, `IN` e `LIKE`.
- A subquery geralmente substitui ou complementa condições em `WHERE`, `HAVING` ou `FROM`.
- Subqueries não podem ser usadas com `ORDER BY`.

### **1.2 Exemplo teórico**
```sql
SELECT <nome_coluna>
FROM <nome_tabela>
WHERE <nome_coluna> <expressao> <operador>  
    ( SELECT <coluna> FROM <tabela> WHERE ... );
```

### **1.3 Exemplo prático**
Queremos encontrar transações realizadas em lojas que cadastraram clientes com valores de compra acima de 160:

```sql
SELECT id_loja, id_cliente, id_transacao 
FROM transacoes 
WHERE id_loja IN
  (SELECT cliente.loja_cadastro 
   FROM cliente 
   WHERE cliente.valor_compra > 160);
```

**Como funciona:**
1. A subquery é executada primeiro:

```sql
SELECT cliente.loja_cadastro 
FROM cliente 
WHERE cliente.valor_compra > 160;
```
**Resultado da subquery:**

| loja_cadastro |
|---------------|
| magalu        |
| postoshell    |
| magalu        |

2. Esse resultado é usado como condição para a main query:

```sql
SELECT id_loja, id_cliente, id_transacao 
FROM transacoes 
WHERE id_loja IN ('magalu', 'postoshell');
```

**Resultado final:**

| id_loja    | id_cliente | id_transacao |
|------------|------------|--------------|
| magalu     | 1          | 768805383    |
| postoshell | 3          | 818770008    |
| magalu     | 1          | 76856563     |

Subqueries podem ser usadas em combinações criativas para resolver problemas complexos de dados!

---

## **2. Agregações por Particionamento**

### **2.1 O que é Particionamento?**
Particionamento é uma organização hierárquica de dados no AWS S3, onde os dados são separados em subpastas com rótulos e valores específicos. 
Isso permite uma otimização significativa no carregamento e consulta dos dados no AWS Athena, reduzindo custos e aumentando a performance.

### **2.2 Como configurar particionamento no AWS S3?**

1. **Estrutura das partições:**
   - Crie uma pasta principal no S3 para armazenar os dados, por exemplo, `transacoes_partition`.
   - Dentro desta pasta, crie subpastas baseadas em rótulos, como o `id_loja`:
     - `transacoes_partition/id_loja=magalu`
     - `transacoes_partition/id_loja=giraffas`
     - `transacoes_partition/id_loja=postoshell`
     - `transacoes_partition/id_loja=subway`
     - ...
   - Em cada subpasta, insira apenas os dados relacionados àquele rótulo (ex.: transações da loja `magalu` na subpasta `id_loja=magalu`).

2. **Criação da tabela particionada no Athena:**

```sql
CREATE EXTERNAL TABLE transacoes_part(
  id_cliente BIGINT, 
  id_transacoes BIGINT, 
  valor DOUBLE) 
  PARTITIONED BY (id_loja string)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES (
  'serialization.format' = ',',
  'field.delim' = ','
) 
LOCATION 's3://transacoes-partition/';
```

3. **Carregar as partições:**

```sql
MSCK REPAIR TABLE transacoes_part;
```

4. **Validar a tabela particionada:**

```sql
SELECT COUNT(*) 
FROM transacoes_part;
```

A partir disso, você pode realizar consultas eficientes utilizando as técnicas aprendidas nos módulos anteriores.

---

## **3. Visões**

### **3.1 O que são Visões?**
Uma visão (“view”) é uma tabela virtual criada com base no resultado de uma consulta. Diferente de uma tabela física, os dados não são armazenados permanentemente, mas gerados dinamicamente toda vez que a visão é referenciada.

### **3.2 Por que utilizar visões?**
- Simplificar consultas complexas, substituindo-as por uma visão reutilizável.
- Criar subconjuntos frequentes dos dados.
- Combinar múltiplas tabelas de forma consistente.
- Reduzir redundância e facilitar o gerenciamento de dados.

### **3.3 Principais funções relacionadas a visões**

| **Comando**          | **Descrição**                                                     |
|-----------------------|-----------------------------------------------------------------|
| `CREATE VIEW`        | Cria uma visão a partir de uma consulta SQL                     |
| `DESCRIBE`           | Lista as colunas e atributos da visão                          |
| `DROP VIEW`          | Exclui uma visão existente                                     |
| `SHOW CREATE VIEW`   | Mostra a query usada para criar a visão                        |
| `SHOW VIEWS`         | Lista todas as visões disponíveis em um banco de dados         |
| `SHOW COLUMNS`       | Lista as colunas presentes em uma visão                        |

### **3.4 Exemplo prático de criação de visão**

```sql
CREATE VIEW clientes_compras_altas AS
SELECT nome, valor_compra, loja_cadastro
FROM cliente
WHERE valor_compra > 1000;
```

**A partir da visão:**

```sql
SELECT * FROM clientes_compras_altas;
```

**Resultado esperado:**

| nome      | valor_compra | loja_cadastro |
|-----------|--------------|---------------|
| joana     | 1300.50      | pontofrio     |

---

Com estas técnicas avançadas, você estará apto a realizar consultas otimizadas e estruturadas, além de organizar seus dados de forma eficiente e reutilizável!
